In [1]:
from ephys_queries import (
    select_neurons
)
from ephys_queries import db_setup_core
from dotenv import load_dotenv

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [29]:
import math

def calculate_distance(ch1, ch2):
    """
    Calculate distances between two channels on a cambridge neurotech 32 channel P series probe.
    
    Electrode spec:
        2 shanks 250um appart
        Each shank has 16 channels in two columns of 8, spaced 22.5um appart
        Contacts are placed 25um above eachother
    """ 
    # Shank
    shank_1 = 1 if ch1 <= 15 else 2
    shank_2 = 1 if ch2 <= 15 else 2
    width = 250 if shank_1 != shank_2 else 0
    
    # Column
    col_1 = 1 if ch1 % 2 == 0 else 2
    col_2 = 1 if ch2 % 2 == 0 else 2
    width = 22.5 if (col_1 != col_2) and (width == 0) else width
    
    #
    ch1t = ch1 - 16 if ch1 > 15 else ch1
    ch2t = ch2 - 16 if ch2 > 15 else ch2
    height = abs(ch1t - ch2t) * 25
    
    return math.hypot(height, width) if width else height

In [55]:
data_dir = Path(".").absolute().parent / "data"
dfb = pd.read_csv(data_dir / "baseline.csv")

In [47]:
df_sc =  pd.read_csv(data_dir / "rsctest_LC_sizes.csv")

In [67]:
df_done = (
    df_sc
    .loc[lambda x: x.binsize==0.2]
    .loc[:, ["spiketrain_1", "spiketrain_2"]]
    .merge(dfb[["neuron_id", "channel"]], left_on="spiketrain_1", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"channel": "st1_chan"})
    .merge(dfb[["neuron_id", "channel"]], left_on="spiketrain_2", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"channel": "st2_chan"})
    .assign(distance= lambda y: y.apply(lambda x: calculate_distance(x.st1_chan, x.st2_chan), axis=1))
)

In [70]:
df_done.head(3)

,spiketrain_1,spiketrain_2,st1_chan,st2_chan,distance
0,1,4,6,17,279.508497
1,1,5,6,20,254.950976
2,4,5,17,20,78.302299


In [71]:
df_done.to_csv(data_dir / "distance.csv", index=False)